# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [36]:
# Import the data
import pandas as pd
import numpy as  np
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split


df = pd.read_csv('Titanic.csv')
df.head(5)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [25]:
# Total number of people who survived/didn't survive
print("Number of people who didn't survive:", np.sum((df['Survived']) == 0))
print("Number of people who survived:", np.sum((df['Survived']) == 1))


Number of people who didn't survive: 549
Number of people who survived: 342


Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [40]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
#columns to create dummy variables for
columns_dummy_variables =  ['Pclass', 'Age', 'SibSp', 'Sex', 'Embarked', 'Survived']
#create dummy variables for Pclass column, SibSp, Sex,Embarked and Survived
Pclass = pd.get_dummies(df['Pclass'],drop_first=True,dtype= 'int')
sex = pd.get_dummies(df['Sex'],drop_first=True,dtype= 'int')
embarked = pd.get_dummies(df['Embarked'],drop_first=True,dtype= 'int')
sibsp = pd.get_dummies(df['SibSp'],drop_first=True,dtype= 'int')

#concatenate the dfs into one main 
dummy_dataframe = pd.concat([df,Pclass,sex,embarked],axis=1)
#drop the sex,pclass,embarked,survived and sibsp
cleaned_dummy_df = dummy_dataframe.drop(['Pclass','Sex','Embarked','SibSp','Name','Parch','Ticket','Cabin','PassengerId'],axis=1)
cleaned_dummy_df


,Survived,Age,Fare,2,3,male,Q,S
0,0,22.0,7.2500,0,1,1,0,1
1,1,38.0,71.2833,0,0,0,0,0
2,1,26.0,7.9250,0,1,0,0,1
3,1,35.0,53.1000,0,0,0,0,1
4,0,35.0,8.0500,0,1,1,0,1
...,...,...,...,...,...,...,...,...
886,0,27.0,13.0000,1,0,1,0,1
887,1,19.0,30.0000,0,0,0,0,1
888,0,NaN,23.4500,0,1,0,0,1
889,1,26.0,30.0000,0,0,1,0,0


Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [41]:
#check the missing values
cleaned_dummy_df.isnull().sum().sort_values(ascending=False)

Age         177
Survived      0
Fare          0
2             0
3             0
male          0
Q             0
S             0
dtype: int64

In [42]:
# Drop missing rows
cleaned_dataframe = cleaned_dummy_df.dropna()
cleaned_dataframe.shape

(714, 8)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [43]:
# Split the data into X and y
y = cleaned_dataframe['Survived']
X = cleaned_dataframe.drop('Survived',axis=1)

In [44]:
#split the training and testing data
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state = 1)

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [45]:
# Build a logistic regression model using statsmodels
log_reg = sm.Logit(y_train, X_train).fit() 

Optimization terminated successfully.
         Current function value: 0.501694
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [49]:
# Summary table
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  499
Model:                          Logit   Df Residuals:                      492
Method:                           MLE   Df Model:                            6
Date:                Wed, 21 Aug 2024   Pseudo R-squ.:                  0.2621
Time:                        00:01:52   Log-Likelihood:                -250.35
converged:                       True   LL-Null:                       -339.28
Covariance Type:            nonrobust   LLR p-value:                 9.644e-36
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Age            0.0050      0.007      0.770      0.442      -0.008       0.018
Fare           0.0146      0.003      4.441      0.000       0.008       0.021
2              0.3439      0.306      1.124      0.2

In [ ]:
# Your comments here
"""
The model is statistically significant because the LLR P-values is less than 0.05. This means the model fits the data well.
But being that the model has a Pseudo R-squ of 0.2621 meaning the model is useful but not highly predictive.
The predictors age, 2,3,Q,S are not statistically significant because their p values are > 0.05. 
Thus the fare and gender are the only predictors necessary for predicting if someone survived or not. Being that the coefficient of fare is 
0.0146 this means that for every unit increase of fare by 1 there is 0.0146 increase in the odds of 
"""



## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [ ]:
# Your code here


In [ ]:
# Your comments here

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!